<a href="https://www.kaggle.com/code/valentindefour/tps-s03e03-eda-automl-fun-explainable-ai?scriptVersionId=116798507" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE

import catboost as cb
from sklearn import model_selection, metrics, linear_model, ensemble, naive_bayes, neighbors, svm, tree
from xgboost import XGBClassifier
import lightgbm

import os
import gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv
/kaggle/input/playground-series-s3e3/sample_submission.csv
/kaggle/input/playground-series-s3e3/train.csv
/kaggle/input/playground-series-s3e3/test.csv


In [2]:
train = pd.read_csv(r'/kaggle/input/playground-series-s3e3/train.csv')
train_o = pd.read_csv(r'/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
test = pd.read_csv(r'/kaggle/input/playground-series-s3e3/test.csv')
ss = pd.read_csv(r'/kaggle/input/playground-series-s3e3/sample_submission.csv')

In [3]:
print(train.shape, train_o.shape, test.shape)

(1677, 35) (1470, 35) (1119, 34)


In [4]:
train.isnull().sum() + train_o.isnull().sum()

Age                         0.0
Attrition                   0.0
BusinessTravel              0.0
DailyRate                   0.0
Department                  0.0
DistanceFromHome            0.0
Education                   0.0
EducationField              0.0
EmployeeCount               0.0
EmployeeNumber              NaN
EnvironmentSatisfaction     0.0
Gender                      0.0
HourlyRate                  0.0
JobInvolvement              0.0
JobLevel                    0.0
JobRole                     0.0
JobSatisfaction             0.0
MaritalStatus               0.0
MonthlyIncome               0.0
MonthlyRate                 0.0
NumCompaniesWorked          0.0
Over18                      0.0
OverTime                    0.0
PercentSalaryHike           0.0
PerformanceRating           0.0
RelationshipSatisfaction    0.0
StandardHours               0.0
StockOptionLevel            0.0
TotalWorkingYears           0.0
TrainingTimesLastYear       0.0
WorkLifeBalance             0.0
YearsAtC

In [5]:
display(train.head())
display(train_o.head())

,id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,...,80,1,10,2,3,10,0,7,8,0
1,1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,...,80,1,4,3,3,4,2,0,3,0
2,2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,...,80,2,4,3,3,3,2,1,2,0
3,3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,...,80,0,15,1,1,6,0,0,2,0
4,4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,...,80,0,31,0,3,31,14,4,10,1


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
display(train['Attrition'].value_counts())
display(train_o['Attrition'].value_counts())

0    1477
1     200
Name: Attrition, dtype: int64

No     1233
Yes     237
Name: Attrition, dtype: int64

In [7]:
train_o['Attrition'] = train_o['Attrition'].map(lambda x: 1 if x == "Yes" else 0)
train_o['id'] = train_o['EmployeeNumber']

In [8]:
train_all = pd.concat([train, train_o.drop('EmployeeNumber', axis = 1)])

In [9]:
display(train_all.shape)
display(train_all['Attrition'].value_counts())

(3147, 35)

0    2710
1     437
Name: Attrition, dtype: int64

In [10]:
target = train_all['Attrition']

data = pd.concat([train_all.drop('Attrition', axis = 1), test]).reset_index(drop = True)

print(train_all.shape, test.shape, data.shape)

(3147, 35) (1119, 34) (4266, 34)


In [11]:
text_features = []

for column in data.columns:
    if data[column].dtype == 'object':
        text_features.append(column)

for text_feature in text_features:
    data = data.join(pd.get_dummies(data[text_feature], prefix=text_feature))
    data = data.drop(text_feature, axis = 1)

In [12]:
data.head()

,id,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0,36,599,24,3,1,4,42,3,1,...,0,0,0,0,0,1,0,1,0,1
1,1,35,921,8,3,1,1,46,3,1,...,0,0,0,1,0,1,0,1,1,0
2,2,32,718,26,3,1,3,80,3,2,...,0,0,1,0,1,0,0,1,1,0
3,3,38,1488,2,3,1,3,40,3,2,...,0,0,0,0,0,1,0,1,1,0
4,4,50,1017,5,4,1,2,37,3,5,...,0,0,0,0,0,0,1,1,0,1


In [13]:
df_train = data.iloc[:len(target), :]

df_test = data.iloc[len(target):, :]

df_train.shape, target.shape, df_test.shape

((3147, 55), (3147,), (1119, 55))

# "MANUAL" ML

In [14]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_train.drop('id', axis = 1), target, random_state = 42)

### CATBOOST

In [15]:
model_cb = cb.CatBoostClassifier(depth = 3,
                                 learning_rate = 0.01,
                                 rsm = 0.5,
                                 random_seed = 42,
                                 verbose = False).fit(X_train, y_train)
print(metrics.roc_auc_score(model_cb.predict(X_test), y_test))
preds_cb = model_cb.predict(df_test.drop('id', axis = 1))
proba_cb = model_cb.predict_proba(df_test.drop('id', axis = 1))

0.8764324324324325


### XGBOOST

In [21]:
clf = XGBClassifier(eval_metric = 'auc')

params_xgb = {'learning_rate': [0.1, 0.05, 0.03],
         'n_estimators': np.arange(100, 500,50),
         'max_depth': [3,4,5],
         'lambda': [1,2,3,4,5],
         'colsample_bytree': [0.3,0.4,0.5,0.6,0.8],
         'subsample': [0.3,0.4,0.5,0.6,0.8],
         'scale_pos_weight': [4,5,6.2,7]
}

rs = model_selection.RandomizedSearchCV(clf, params_xgb, n_iter = 500, n_jobs = -1, cv = 3, scoring = 'roc_auc',
                       verbose = 1)
rs.fit(X_train, y_train)
print(rs.best_score_)
print(rs.best_params_)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits
0.8496099576741768
{'subsample': 0.4, 'scale_pos_weight': 4, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'lambda': 4, 'colsample_bytree': 0.5}


In [23]:
model_xgb = XGBClassifier(subsample = 0.4, scale_pos_weight = 4 , n_estimators = 200, max_depth = 3, learning_rate = 0.05, colsample_bytree = 0.5, eval_metric = "auc").fit(X_train, y_train)

print(metrics.roc_auc_score(model_xgb.predict(X_test), y_test))

proba_xgb = model_cb.predict_proba(df_test.drop('id', axis = 1))

0.737034517020909


### LIGHTGBM

In [17]:
model_lgbm = lightgbm.LGBMClassifier().fit(X_train, y_train)
print(metrics.roc_auc_score(model_lgbm.predict(X_test), y_test))
preds_lgbm = model_lgbm.predict(df_test.drop('id', axis = 1))
proba_lgbm = model_lgbm.predict_proba(df_test.drop('id', axis = 1))

0.7554751430868927


### BLENDING THEM ALL TOGETHER

In [18]:
proba_global = (proba_cb+proba_xgb+proba_lgbm)/3
proba_global

array([[0.73188835, 0.26811165],
       [0.87269235, 0.12730765],
       [0.92160381, 0.07839619],
       ...,
       [0.96059179, 0.03940821],
       [0.96705772, 0.03294228],
       [0.98617029, 0.01382971]])

### TRYING A SUBMISSION

In [ ]:
ss[['Stay','Attrition']] = proba_global
ss[['id','Attrition']].to_csv('submission.csv', index = False)

This submission got a public score of
## 0.92514.

---

### Let's now try with h2o Auto ML :

# AUTO ML

## Init cluster and data loading

In [20]:
import h2o        
from h2o.automl import H2OAutoML

In [21]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp31rox_jv
  JVM stdout: /tmp/tmp31rox_jv/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp31rox_jv/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_unknownUser_2231j5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [22]:
target.to_frame().reset_index(drop = True)

,Attrition
0,0
1,0
2,0
3,0
4,1
...,...
3142,0
3143,0
3144,0
3145,0


In [23]:
df_train_h2o = df_train.copy()
df_train_h2o['Attrition'] = target.to_frame().reset_index(drop = True)
train_h2o = h2o.H2OFrame(df_train_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [24]:
test_h2o = h2o.H2OFrame(df_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


## Training

In [25]:
train, test = train_h2o.split_frame(ratios = [0.75])

x = train.columns
y = 'Attrition'
x.remove(y)

In [26]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models - or 10 min
aml = H2OAutoML(max_models=20,
                max_runtime_secs=600,
                seed=42)

aml.train(x=x, y=y, training_frame=train)

AutoML progress: |
13:53:28.773: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

██
13:53:34.120: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]


13:53:37.553: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

██
13:53:44.81: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

█
13:53:46.984: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

██
13:53:53.5: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

█
13:53:56.459: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

█
13:53:59.780: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

█
13:54:03.329: _train param, Dropping bad and constant columns: [StandardHours, EmployeeCount, Over18_Y]

█
13:54:05.320: _trai

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_1_AutoML_1_20230119_135328

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0594680030247141
RMSE: 0.24386062212812076
LogLoss: 0.20856837857198565
AUC: 0.9402360641124854
AUCPR: 0.7864848360235648
Gini: 0.8804721282249708
Null degrees of freedom: 2361
Residual degrees of freedom: 2357
Null deviance: 1877.571265042212
Residual deviance: 985.2770203740603
AIC: 995.2770203740603

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3248061760044085
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      1964  77   0.0377   (77.0/2041.0)
1      92    229  0.2866   (92.0/321.0)
Total  2056  306  0.0715   (169.0/2362.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.324806     0.730463  155
max f2                       0.18886      0.772853  221
max f0point5                 0.387309     0.76839   135
max accuracy                 0.387309     0.931414  135
max precision                0.954025     1         0
max recall                   0.018013     1         373
max specificity              0.954025     1         0
max absolute_mcc             0.324806     0.689502  155
max min_per_class_accuracy   0.181666     0.872274  225
max mean_per_class_accuracy  0.18886      0.875295  221
max tns                      0.954025     2041      0
max fns                      0.954025     320       0
max fps                      0.0018588    2041      399
max tps                      0.018013     321       373
max tnr                      0.954025     1         0
max fnr                      0.954025     0.996885  0
max fpr                      0.0018588    1         399
max tpr                      0.018013     1         373

Gains/Lift Table: Avg response rate: 13.59 %, avg score: 13.88 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101609                   0.838448           7.35826    7.35826            1                0.886906    1                           0.886906            0.0747664       0.0747664                  635.826   635.826            0.0747664
2        0.0203218                   0.767092           7.05166    7.20496            0.958333         0.805406    0.979167                    0.846156            0.0716511       0.146417                   605.166   620.496            0.145927
3        0.0300593                   0.710691           6.39848    6.94371            0.869565         0.74284     0.943662                    0.812688            0.0623053       0.208723                   539.848   594.371            0.206763
4        0.0402202                   0.662648           6.43847    6.81607            0.875            0.686367    0.926316                    0.780775            0.0654206       0.274143                   543.847   581.607            0.270714
5        0.050381                    0.619076           6.43847    6.73991            0.875            0.633828    0.915966                    0.751139            0.0654206       0.339564                   543.847   573.991            0.334664
6        0.100339                    0.415541           5.30044    6.02321            0.720339         0.514274    0.818565                    0.633206            0.264798        0.604361                  

In [28]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20230119_135328,0.841846,0.293385,0.532048,0.265982,0.293045,0.0858754
StackedEnsemble_AllModels_1_AutoML_1_20230119_135328,0.841247,0.292957,0.527422,0.270234,0.294163,0.0865316
GBM_1_AutoML_1_20230119_135328,0.830452,0.301631,0.512135,0.245272,0.299028,0.0894176
GLM_1_AutoML_1_20230119_135328,0.829267,0.302288,0.506378,0.24333,0.298017,0.0888141
XGBoost_1_AutoML_1_20230119_135328,0.822866,0.310898,0.475367,0.257751,0.304159,0.0925129
GBM_2_AutoML_1_20230119_135328,0.821469,0.311137,0.483572,0.295665,0.303615,0.0921818
GBM_5_AutoML_1_20230119_135328,0.814886,0.317166,0.470722,0.265923,0.306605,0.0940064
GBM_4_AutoML_1_20230119_135328,0.81292,0.316315,0.468551,0.289291,0.305992,0.0936312
GBM_grid_1_AutoML_1_20230119_135328_model_1,0.811006,0.318359,0.458544,0.291741,0.307198,0.0943707
XGBoost_3_AutoML_1_20230119_135328,0.810861,0.323104,0.465859,0.279295,0.306945,0.0942155


In [31]:
preds = aml.leader.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [34]:
ss['Attrition'] = preds.as_data_frame()['p1']
ss.to_csv('submission.csv', index = False)